## Problem Statement

Q9. Design compiler for the following hypothetical languages

In [254]:
%%writefile question.txt
int main()
begin
 int n;
 do
  expr=expr+expr;
  n=exp;
 while(exp)
 return(n)
end

Overwriting question.txt


## Lexical Analyzer

In [255]:
import re
from tabulate import tabulate

delimiters=[" ","+","-","*","/","=",";","(",")","[","]","{","}","<",">","!","&","|","^","%","~","?",".",",","'","\""]
keywords=['int','main','begin','end','do','while','return']

kwd_dict={
    "int":"t",
    "main":"m()",
    "begin":"b",
    "end":"d",
    "do":"do",
    "while":"w",
    "return":"r",
    "+":"o",
    "-":"o",
    "*":"o",
    "/":"o",
    "=":"a",
    "expr":"e",
    "exp":"e",
    "n":"id"
}

def isKeyword(token):
    if token in keywords:
        return True
    return False

def isDelimiter(ch):
    if ch in delimiters:
        return True
    return False

print("------------------------ LEXICAL ANALYZER - TOKENIZER -----------------------")
tokentable_global=[]
tokentable_global.append(["Token No","Lexeme","Token","Line No"])
txt=open("question.txt","r")
tokens=txt.read()
count=0
tkncount=0
delimit_flag=0
program = tokens.split("\n")

for line in program:
    err=0
    prevct=tkncount
    count = count + 1
    tokentable_local=[]
    print(f"At line {count}\nContent: {line}\n")

    tokens=line
    tokens=re.findall(r"[A-Za-z0-9_]+|[0-9]+|[(){}]|\S", tokens)

    print("Tokens found: ")
    tokentable=[]
    tokentable.append(["Lexeme","Token"])
    for token in tokens:
        if isDelimiter(token):
            if token in ["{","}","(",")",";",","]:
                tkncount+=1
                tokentable.append([token,"Delimiter"])
                tokentable_local.append([tkncount,token,"Delimiter",count])
                
            elif token in ["+","-","*","/","="]:
              if token in ["+","-","*","/"]:
                tkncount+=1
                tokentable.append([token,"Arithmetic Operator"])
                tokentable_local.append([tkncount,token,"Arithmetic Operator",count])
                
              elif token in ["="]:
                tkncount+=1
                tokentable.append([token,"Assignment Operator"])
                tokentable_local.append([tkncount,token,"Assignment Operator",count])
                
            else:
                tokentable.append([token,"Invalid Character [Error]"])
                print("Error Recovery: Line Ignored")
                err=1
                break
            continue
        else:

            if isKeyword(token):
                tkncount+=1
                tokentable.append([token,"Keyword"])
                tokentable_local.append([tkncount,token,"Keyword",count])
                
            else:
                if token.isnumeric():
                    tkncount+=1
                    tokentable.append([token,"Number"])
                    tokentable_local.append([tkncount,token,"Number",count])
                    
                else:        
                    if re.match("^[a-zA-Z][a-zA-Z0-9_]*", token) is not None:
                        tkncount+=1
                        tokentable.append([token,"Identifier"])
                        tokentable_local.append([tkncount,token,"Identifier",count])

                    else:
                        tokentable.append([token,"Invalid Character [Error]"])
                        print("Error Recovery: Line Ignored")
                        err=1
                        break
    delimit_flag=0                    
    if err != 1:
      for entry in tokentable_local:
        tokentable_global.append(entry)
    else:
      tkncount=prevct

    print(tabulate(tokentable, headers="firstrow", tablefmt="grid"))
    print("\n----------------------------------------------------")

print("\nGlobal Token Table: ")
print(tabulate(tokentable_global, headers="firstrow", tablefmt="grid"))

# For the parser tool
mth_flag=0
with open('tokens.txt', 'w') as f:
    str_to_load=""
    for token in tokentable_global[1:]:
        if mth_flag > 0:
            mth_flag-=1
            continue
        sym=kwd_dict.get(token[1]) if kwd_dict.get(token[1]) is not None else token[1]
        str_to_load+=str(sym)+" "
        if token[1] == "main":
            mth_flag=2
    f.write(f"{str_to_load}\n")

------------------------ LEXICAL ANALYZER - TOKENIZER -----------------------
At line 1
Content: int main()

Tokens found: 
+----------+-----------+
| Lexeme   | Token     |
+==========+===========+
| int      | Keyword   |
+----------+-----------+
| main     | Keyword   |
+----------+-----------+
| (        | Delimiter |
+----------+-----------+
| )        | Delimiter |
+----------+-----------+

----------------------------------------------------
At line 2
Content: begin

Tokens found: 
+----------+---------+
| Lexeme   | Token   |
+==========+=========+
| begin    | Keyword |
+----------+---------+

----------------------------------------------------
At line 3
Content:  int n;

Tokens found: 
+----------+------------+
| Lexeme   | Token      |
+==========+============+
| int      | Keyword    |
+----------+------------+
| n        | Identifier |
+----------+------------+
| ;        | Delimiter  |
+----------+------------+

----------------------------------------------------
At lin

## Parser

Defining functions for the parser

In [256]:
import re
from tabulate import tabulate

def removeLeftRecursion(rulesDiction):
    store = {}
    for lhs in rulesDiction:
        alphaRules = []
        betaRules = []
        allrhs = rulesDiction[lhs]
        for subrhs in allrhs:
            if subrhs[0] == lhs:
                alphaRules.append(subrhs[1:])
            else:
                betaRules.append(subrhs)
        if len(alphaRules) != 0:
            lhs_ = lhs + "'"
            while (lhs_ in rulesDiction.keys()) \
                    or (lhs_ in store.keys()):
                lhs_ += "'"
            for b in range(0, len(betaRules)):
                betaRules[b].append(lhs_)
            rulesDiction[lhs] = betaRules
            for a in range(0, len(alphaRules)):
                alphaRules[a].append(lhs_)
            alphaRules.append(['#'])
            store[lhs_] = alphaRules
    for left in store:
        rulesDiction[left] = store[left]
    return rulesDiction


def LeftFactoring(rulesDiction):
    newDict = {}
    for lhs in rulesDiction:
        allrhs = rulesDiction[lhs]
        temp = dict()
        for subrhs in allrhs:
            if subrhs[0] not in list(temp.keys()):
                temp[subrhs[0]] = [subrhs]
            else:
                temp[subrhs[0]].append(subrhs)
        new_rule = []
        tempo_dict = {}
        for term_key in temp:
            allStartingWithTermKey = temp[term_key]
            if len(allStartingWithTermKey) > 1:
                lhs_ = lhs + "'"
                while (lhs_ in rulesDiction.keys()) \
                        or (lhs_ in tempo_dict.keys()):
                    lhs_ += "'"
                new_rule.append([term_key, lhs_])
                ex_rules = []
                for g in temp[term_key]:
                    ex_rules.append(g[1:])
                tempo_dict[lhs_] = ex_rules
            else:
                new_rule.append(allStartingWithTermKey[0])
        newDict[lhs] = new_rule
        for key in tempo_dict:
            newDict[key] = tempo_dict[key]
    return newDict


def first(rule):
    global rules, nonterm_userdef, \
        term_userdef, diction, firsts
    if len(rule)!= 0 and (rule is not None):
        if rule[0] in term_userdef:
            return rule[0]
        elif rule[0] == '#':
            return '#'

    if len(rule)!= 0:
        if rule[0] in list(diction.keys()):
            fres = []
            rhs_rules = diction[rule[0]]
            for itr in rhs_rules:
                indivRes = first(itr)
                if type(indivRes) is list:
                    for i in indivRes:
                        fres.append(i)
                else:
                    fres.append(indivRes)

            if '#' not in fres:
                return fres
            else:
                newList = []
                fres.remove('#')
                if len(rule) > 1:
                    ansNew = first(rule[1:])
                    if ansNew != None:
                        if type(ansNew) is list:
                            newList = fres + ansNew
                        else:
                            newList = fres + [ansNew]
                    else:
                        newList = fres
                    return newList
                fres.append('#')
                return fres


def follow(nt):
    global start_symbol, rules, nonterm_userdef,term_userdef, diction, firsts, follows

    solset = set()
    if nt == start_symbol:
        solset.add('$')

    for curNT in diction:
        rhs = diction[curNT]
        for subrule in rhs:
            if nt in subrule:
                while nt in subrule:
                    index_nt = subrule.index(nt)
                    subrule = subrule[index_nt + 1:]
                    if len(subrule) != 0:
                        res = first(subrule)
                        if '#' in res:
                            newList = []
                            res.remove('#')
                            ansNew = follow(curNT)
                            if ansNew != None:
                                if type(ansNew) is list:
                                    newList = res + ansNew
                                else:
                                    newList = res + [ansNew]
                            else:
                                newList = res
                            res = newList
                    else:
                        if nt != curNT:
                            res = follow(curNT)

                    if res is not None:
                        if type(res) is list:
                            for g in res:
                                solset.add(g)
                        else:
                            solset.add(res)
    return list(solset)


def computeAllFirsts():
    global rules, nonterm_userdef, term_userdef, diction, firsts
    for rule in rules:
        k = rule.split("->")
        k[0] = k[0].strip()
        k[1] = k[1].strip()
        rhs = k[1]
        multirhs = rhs.split('|')
        for i in range(len(multirhs)):
            multirhs[i] = multirhs[i].strip()
            multirhs[i] = multirhs[i].split()
        diction[k[0]] = multirhs

    print(f"\nRules: \n")
    for y in diction:
        print(f"{y}->{diction[y]}")

    diction = removeLeftRecursion(diction)

    diction = LeftFactoring(diction)

    for y in list(diction.keys()):
        t = set()
        for sub in diction.get(y):
            res = first(sub)
            if res != None:
                if type(res) is list:
                    for u in res:
                        t.add(u)
                else:
                    t.add(res)

        firsts[y] = t

    print("\nCalculated firsts: ")
    key_list = list(firsts.keys())
    index = 0
    for gg in firsts:
        print(f"first({key_list[index]}) "
              f"=> {firsts.get(gg)}")
        index += 1

def computeAllFollows():
    global start_symbol, rules, nonterm_userdef,\
        term_userdef, diction, firsts, follows
    for NT in diction:
        solset = set()
        sol = follow(NT)
        if sol is not None:
            for g in sol:
                solset.add(g)
        follows[NT] = solset

    print("\nCalculated follows: ")
    key_list = list(follows.keys())
    index = 0
    for gg in follows:
        print(f"follow({key_list[index]})"
              f" => {follows[gg]}")
        index += 1


def createParseTable():
    import copy
    global diction, firsts, follows, term_userdef
    
    print("\nFirsts and Follow Result table\n")

    mx_len_first = 0
    mx_len_fol = 0
    for u in diction:
        k1 = len(str(firsts[u]))
        k2 = len(str(follows[u]))
        if k1 > mx_len_first:
            mx_len_first = k1
        if k2 > mx_len_fol:
            mx_len_fol = k2

    firstfollow_table=[]
    firstfollow_table.append(["Non-Terminal","First","Follow"])
    
    for u in diction:
        firstfollow_table.append([u,str(firsts[u]),str(follows[u])])
        
    print(tabulate(firstfollow_table, headers="firstrow", tablefmt="grid"))

    ntlist = list(diction.keys())
    terminals = copy.deepcopy(term_userdef)
    terminals.append('$')

    mat = []
    for x in diction:
        row = []
        for y in terminals:
            row.append('')
        mat.append(row)

    grammar_is_LL = True

    for lhs in diction:
        rhs = diction[lhs]
        for y in rhs:
            res = first(y)
            if '#' in res:
                if type(res) == str:
                    firstFollow = []
                    fol_op = follows[lhs]
                    if fol_op is str:
                        firstFollow.append(fol_op)
                    else:
                        for u in fol_op:
                            firstFollow.append(u)
                    res = firstFollow
                else:
                    res.remove('#')
                    res = list(res) +\
                        list(follows[lhs])
            ttemp = []
            if type(res) is str:
                ttemp.append(res)
                res = copy.deepcopy(ttemp)
            for c in res:
                xnt = ntlist.index(lhs)
                yt = terminals.index(c)
                if mat[xnt][yt] == '':
                    mat[xnt][yt] = mat[xnt][yt] \
                        + f"{lhs}->{' '.join(y)}"
                else:
                    if f"{lhs}->{y}" in mat[xnt][yt]:
                        continue
                    else:
                        grammar_is_LL = False
                        mat[xnt][yt] = mat[xnt][yt] \
                            + f",{lhs}->{' '.join(y)}"

    parse_table=[]
    print("\nGenerated parsing table:\n")
    
    frmt = "{:>15}" * len(terminals)
    parse_table.append([""]+terminals)
    
    for i in range(len(mat)):
        parse_table.append([ntlist[i]]+mat[i])

    print(tabulate(parse_table, headers="firstrow", tablefmt="grid"))
    
    return (mat, grammar_is_LL, terminals)


def validateStringUsingStackBuffer(parsing_table, grammarll1,
                                   table_term_list, input_string,
                                   term_userdef, start_symbol):


    print(f"\nValidating program:\nOutput from Lexer: \n{input_string}\n")

    if grammarll1 == False:
        return f"\nInput String = " \
            f"\"{input_string}\"\n" \
            f"Grammar is not LL(1)"

    stack = [start_symbol, '$']
    buffer = []

    input_string = input_string.split()
    input_string.reverse()
    buffer = ['$'] + input_string

    eval_table=[]
    eval_table.append(["Input","Stack","Action"])

    print("Evaluating Program Code: ")

    err=0
    while True:
        if stack == ['$'] and buffer == ['$']:
            eval_table.append([' '.join(buffer),
                               ' '.join(stack),
                               "Valid"])
            print(tabulate(eval_table, headers="firstrow", tablefmt="grid"))  
            return "\nValid String!"
        elif stack[0] not in term_userdef:
            x = list(diction.keys()).index(stack[0])
            y = table_term_list.index(buffer[-1])
            if parsing_table[x][y] != '':
                entry = parsing_table[x][y]
                eval_table.append([' '.join(buffer),
                                   ' '.join(stack),
                                    f"T[{stack[0]}][{buffer[-1]}] = {entry}"])
            
                lhs_rhs = entry.split("->")
                lhs_rhs[1] = lhs_rhs[1].replace('#', '').strip()
                entryrhs = lhs_rhs[1].split()
                stack = entryrhs + stack[1:]
            else:
                # Panic mode recovery
                eval_table.append([' '.join(buffer),
                                   ' '.join(stack),
                              f"\nNo rule at " \
                    f"Table[{stack[0]}][{buffer[-1]}]. [Skipped]"])
                stack = stack[1:]
                
        else:
            if stack[0] == buffer[-1]:
                eval_table.append([' '.join(buffer),
                                   ' '.join(stack),
                              f"Matched:{stack[0]}"])
                
                buffer = buffer[:-1]
                stack = stack[1:]
            else:
                # Panic mode recovery
                eval_table.append([' '.join(buffer),
                                   ' '.join(stack),
                              f"Unmatched terminal symbol [Skipped]"])
                if err==0:
                    err+=1
                    stack = stack[1:]
                else:
                    err-=1
                    buffer = buffer[:-1]               
    


Main function

In [257]:
sample_input_string = ""

rules=["S -> A B",
       "A -> t m()",
       "B -> b M W r ( id ) d",
       "M -> t id ; | #",
       "E1 -> e a e o e ; | #",
       "E2 -> id a e ; | #",
       "W -> do E1 E2 w ( e ) | #"]

nonterm_userdef=['S','A','B','M','E1','E2','W']
term_userdef=['t','m()','b','d','r','e','o',';','id','a','do','w','(',')']

sample_input_string=str_to_load

#Sending output to parser from a file (generated by lexer)
#inps = ''
#with open('tokens.txt', 'r+') as f:
#    for line in f.readlines():
#        inps += line
#sample_input_string = inps

diction = {}
firsts = {}
follows = {}

computeAllFirsts()

start_symbol = list(diction.keys())[0]

computeAllFollows()

(parsing_table, result, tabTerm) = createParseTable()

if sample_input_string != None:
    validity = validateStringUsingStackBuffer(parsing_table, result,
                                              tabTerm, sample_input_string,
                                              term_userdef, start_symbol)
    print(validity)

else:
    print("\nNo input string detected")


Rules: 

S->[['A', 'B']]
A->[['t', 'm()']]
B->[['b', 'M', 'W', 'r', '(', 'id', ')', 'd']]
M->[['t', 'id', ';'], ['#']]
E1->[['e', 'a', 'e', 'o', 'e', ';'], ['#']]
E2->[['id', 'a', 'e', ';'], ['#']]
W->[['do', 'E1', 'E2', 'w', '(', 'e', ')'], ['#']]

Calculated firsts: 
first(S) => {'t'}
first(A) => {'t'}
first(B) => {'b'}
first(M) => {'t', '#'}
first(E1) => {'e', '#'}
first(E2) => {'id', '#'}
first(W) => {'do', '#'}

Calculated follows: 
follow(S) => {'$'}
follow(A) => {'b'}
follow(B) => {'$'}
follow(M) => {'do', 'r'}
follow(E1) => {'id', 'w'}
follow(E2) => {'w'}
follow(W) => {'r'}

Firsts and Follow Result table

+----------------+-------------+-------------+
| Non-Terminal   | First       | Follow      |
+================+=============+=============+
| S              | {'t'}       | {'$'}       |
+----------------+-------------+-------------+
| A              | {'t'}       | {'b'}       |
+----------------+-------------+-------------+
| B              | {'b'}       | {'$'}       |
+

## End of notebook

<i>For Compiler Design - 19CS702</i>